In [27]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import AnimalShelter class 
from animal_shelter_crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Instantiate the AnimalShelter class with the required credentials

username = "aacuser"
password = "paulinaWeaver"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
if '_id' in df.columns:  # Check if the column exists before dropping it to prevent KeyError
    df.drop(columns=['_id'], inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    
    # Dashboard title
    html.Center(html.B(html.H1('CS-340 Animal Shelter Dashboard'))),
    
    # Logo is centered and linked to SNHU website
    html.Center(
        html.A(
            html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height': '200px'}),
            href='https://www.snhu.edu',  # The client's home page
            target="_blank"  # Opens in new tab
        )
    ),
    # Creator's name centered below the logo
    html.Center(html.H2("Created by: Paulina Weaver", style={'color': 'black', 'marginTop': '15px', 'fontSize': '16px'})),

    # Horizontal line
    html.Hr(),
    
    # This Div contains a label and radio buttons to filter rescue types
    html.Div([
        html.Label("Filter by:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster Rescue or Individual Tracking', 'value': 'disaster'},
                {'label': 'Reset', 'value': 'reset'}
            ],
            value='reset',
            labelStyle={'display': 'block'}
        )
    ]),

    # Horizontal line
    html.Hr(),
    
    # Configure the Dash DataTable to improve usability for the client
    # Set up the DataTable with read-only, multi-column sorting, single row/column selection, 
    # pagination, and filtering for enhanced usability
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10,             
    ),
    
    # Line break
    html.Br(),
    
    # Horizontal line
    html.Hr(),
    
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])

def update_dashboard(filter_type):
    # Default empty query
    query = {}

    # Filter for Water Rescue scenario
    if filter_type == 'water':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": [
                "Labrador Retriever Mix",
                "Chesapeake Bay Retriever",
                "Newfoundland"
            ]},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    # Filter for Mountain or Wilderness Rescue scenario
    elif filter_type == 'mountain':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": [
                "German Shepherd",
                "Alaskan Malamute",
                "Old English Sheepdog",
                "Siberian Husky",
                "Rottweiler"
            ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}
        }

    # Filter for Disaster Rescue or Individual Tracking scenario
    elif filter_type == 'disaster':
        query = {
            "animal_type": "Dog",
            "breed": {"$in": [
                "Doberman Pinscher",
                "German Shepherd",
                "Golden Retriever",
                "Bloodhound",
                "Rottweiler"
            ]},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}
        }

    # Reset filter to show all data
    elif filter_type == 'reset':
        query = {}

    # Query MongoDB with the filter
    filtered_data = db.read(query)
    
    # Convert the data into a pandas DataFrame
    dff = pd.DataFrame.from_records(filtered_data)

    # Remove the '_id' column if it exists
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True)

    # Return the data as a list of dictionaries
    return dff.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])

def update_graphs(viewData):
    # Check if viewData is empty or None
    if not viewData:
        # If no data is available, display a message
        return [html.Div("No data available to display the chart.")]

    # Convert filtered data to DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Check if the DataFrame is empty, display a message
    if dff.empty:
        return [html.Div("No data available to display the chart.")]

    # Create a pie chart to show the overview of dog breeds
    fig = px.pie(dff, names='breed', title='Dog Breed Overview')

    # Return the graph 
    return [
        dcc.Graph(figure=fig)
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)

def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

# Callback function to update the map based on selected data table row
def update_map(viewData, index):  
    # Return a message if no data is available
    if not viewData:
        return [html.Div("Please select an animal to display its location on the map.")]

     # Convert the viewData into a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    try:
        # Use selected row index if available, otherwise default to row 0
        row = index[0] if index else 0
        
        # Retrieve location and animal info from the selected row
        lat = dff.iloc[row]["location_lat"]
        long = dff.iloc[row]["location_long"]
        breed = dff.iloc[row]["breed"]
        name = dff.iloc[row]["name"]

        # Return a Dash Leaflet map centered on the selected animal's location
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, long], zoom=10, children=[              
                # Add map tiles
                dl.TileLayer(),
                # Add a marker at the location
                dl.Marker(position=[lat, long], children=[
                    # Tooltip showing breed
                    dl.Tooltip(breed),
                    # Popup with animal info
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(name)
                    ])
                ])
            ])
        ]
    
    except Exception as e:
        # Print error for debugging
        print("Map error:", e)
        return [html.Div("Unable to display map.")]


app.run_server(debug=True)


Dash app running on http://127.0.0.1:27683/
